## astropy.modeling solutions

### Exercise 1:

#Generate fake data
```
np.random.seed(0)
x = np.linspace(-5., 5., 200)
y = 3 * np.exp(-0.5 * (x - 1.3)**2 / 0.8**2)
y += np.random.normal(0., 0.2, x.shape)
```
- Fit the data with a Trapezoid1D model.
- Fit a Gaussian1D model to it.
- Display the results.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

from astropy.modeling import models, fitting

Create the data.

In [ ]:
np.random.seed(0)
x = np.linspace(-5., 5., 200)
y = 3 * np.exp(-0.5 * (x - 1.3)**2 / 0.8**2)
y += np.random.normal(0., 0.2, x.shape)
plt.plot(x, y)

Let's see what the parameter names for Trapezoi1D are.

In [ ]:
print(models.Trapezoid1D.param_names)

Create a Trapezoid1D model - the prameter values will be used as initial guesses for the fitting.

In [ ]:
trapez_init = models.Trapezoid1D(amplitude=1., x_0=0., width=1., slope=0.5)

Fit the model to the data. The output is a new model - `trapez`.

In [ ]:
fitter = fitting.LevMarLSQFitter()
trapez = fitter(trapez_init, x, y)
print(trapez.parameters)

In a similar way fit a Gaussian to the data.

In [ ]:
gauss_init = models.Gaussian1D(amplitude=1, mean=0, stddev=1)
gauss = fitter(gauss_init, x, y)
print(gauss.parameters)

Diplay the results.

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(x, y, 'ko', label='data')
plt.plot(x, trapez(x), 'r', label='Trapezoid', lw=3)
plt.plot(x, gauss(x), 'g', label='Gaussian', lw=3)
plt.legend(loc=2)

Exercise 2:

- read a spectrum from a text file (data/sample.txt)
- Using the rest wavelengths as initial values, fit a gaussian to the H beta and OIII lines.


In [ ]:
from astropy.io import ascii
sdss = ascii.read('sample_sdss.txt')
sdss.colnames

In [ ]:
wave = sdss['lambda']
flux = sdss['flux']
#%matplotlib notebook
plt.plot(wave, flux)

Use the rest wavelengths as initial values for the locaiton of the lines.


In [ ]:
Hbeta = 4862.721
Halpha = 6564.614
OIII_1 = 4958.911
OIII_2 = 5008.239
Na = 6549.86
Nb = 6585.27
Sa = 6718.29
Sb = 6732.68

In [ ]:
wave = sdss['lambda']
flux = sdss['flux']
plt.figure(figsize=(8, 5))
plt.plot(wave, flux)
plt.text(4800, 70, 'Hbeta', rotation=90)
plt.text(4900, 100, 'OIII_1', rotation=90)
plt.text(4950, 200, 'OIII_2', rotation=90)
plt.text(6500, 170, 'Halpha', rotation=90)
plt.text(6700, 70, 'Sa and Sb', rotation=90)

Create a Polynomial model to fit the continuum.

In [ ]:
mean_flux = flux.mean()
cont = np.where(flux > mean_flux, mean_flux, flux)
linfitter = fitting.LinearLSQFitter()
poly_cont = linfitter(models.Polynomial1D(1), wave, cont)
print(poly_cont)

Create Gaussin1D models for each of the Hbeta and OIII lines.

In [ ]:
h_beta = models.Gaussian1D(amplitude=34, mean=Hbeta, stddev=5)
o3 = models.Gaussian1D(amplitude=170, mean=OIII_2, stddev=5)
o1 = models.Gaussian1D(amplitude=57, mean=OIII_1, stddev=5)

Create a compound model for the three lines and the continuum.

In [ ]:
hbeta_combo = h_beta + o1 + o3 + poly_cont
print(hbeta_combo.param_names)

Tie the ratio of the intensity of the two OIII lines.

In [ ]:
def tie_ampl(model):
    return model.amplitude_2 / 3.1

hbeta_combo.amplitude_1.tied = tie_ampl

Also tie the wavelength of the Hbeta line to the OIII wavelength.

In [ ]:
def tie_wave(model):
    return model.mean_0 * OIII_1/Hbeta

hbeta_combo.mean_1.tied = tie_wave 

Fit all lines simultaneously.

In [ ]:
fitter = fitting.LevMarLSQFitter()
fitted_model = fitter(hbeta_combo, wave, flux)
fitted_lines = fitted_model(wave)

In [ ]:
plt.figure(figsize=(9, 6))
plt.plot(wave, flux)
plt.plot(wave, fitted_lines, 'r')